In [ ]:
#This notebook is by Anastasia Ruzmaikina for Kaggle Competition Contradictory, My Dear Watson

If you have two sentences, there are three ways they could be related: one could entail the other, one could contradict the other, or they could be unrelated. Natural Language Inferencing (NLI) is a popular NLP problem that involves determining how pairs of sentences (consisting of a premise and a hypothesis) are related.

Your task is to create an NLI model that assigns labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction) to pairs of premises and hypotheses. To make things more interesting, the train and test set include text in fifteen different languages!

In this notebook, I use HuggingFace Deberta-V3-Multi to analyze how pairs of sentences are related. This notebook has 67.1% accuracy on the competition test dataset.

In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_core as keras
import keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import requests
from keras.activations import softmax

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df1 = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
df2 = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
df = df1 #pd.concat([df1, df2], axis=0)
df= df.sample(frac =0.25)
df_train = df.reset_index()
df_train.drop(columns=['index'], inplace=True)
print(df_train.head())
df_test = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")
df_test.head()

from datasets import load_dataset
#ds = load_dataset('csv', data_files=['/kaggle/input/llm-detect-ai-generated-text/train_essays.csv', '/kaggle/input/generated2/generated.csv', '/kaggle/input/generated1/generated1.csv'])
#print(ds)
print(len(df_train))
print(len(df_test))

df_train['premise'] = df_train['premise'].str.lower()
#df_train.set_index('id', inplace=True)
#df_train['generated'] = df_train['generated'].map({1:'yes', 0:'no'})
#df_train['prompt_id'] = df_train['prompt_id'].map({1:'electoral', 0:'cars'})
df_train['premise'] = df_train['premise'].str.replace("#", "")
#df_train = df_train.sample(frac=0.8)
df_test['premise'] = df_test['premise'].str.replace("#", "" )
df_test['premise'] = df_test['premise'].str.lower()
df_train['hypothesis'] = df_train['hypothesis'].str.lower()
df_train['hypothesis'] = df_train['hypothesis'].str.replace("#", "")
df_test['hypothesis'] = df_test['hypothesis'].str.replace("#", "" )
df_test['hypothesis'] = df_test['hypothesis'].str.lower()

#df_train = df_train.sample(frac=0.8)
#df_test['generated_text'] = df_test['generated'].map({1:'yes', 0:'no'})
#df_test['prompt_id'] = df_test['prompt_id'].map({1:'electoral', 0:'cars'})

from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict
df_train['input'] = 'TEXT1: '+ df_train.premise + ';  TEXT2: '+ df_train.hypothesis + ';'
#'TEXT: ' + df_train.text + ';  ANC: '+ df_train.prompt_id# + '; ANC2: '+ df_train.id
#TEXT2: ' + df_train.generated + ';
df_train1 = df_train.drop(['id', 'lang_abv'], axis=1)#, 'language'
ds = Dataset.from_pandas(df_train1)
print(ds)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from transformers import DebertaV3Model
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import TextClassificationPipeline, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments,Trainer
model_nm = '/kaggle/input/huggingface-deberta-variants/deberta-base-mnli/deberta-base-mnli'  #'/kaggle/input/debertav3small'       #'/kaggle/input/debertav3small'
tokz = AutoTokenizer.from_pretrained(model_nm)
def tok_func(x): return tokz(x["input"])
tok_ds = ds.map(tok_func, batched=True)
tok_ds = tok_ds.rename_columns({'label':'labels'})
dds = tok_ds.train_test_split(0.15, seed=420)
print(dds)
df_test['input'] = 'TEXT1: ' + df_test.premise + '; TEXT2: ' +df_test.hypothesis + ';' #'TEXT: ' + df_test.text + '; ANC: ' + df_test.prompt_id#+ '; ANC2: '+ df_test.id
df_test1 = df_test.drop(['id', 'lang_abv'], axis=1)
eval_ds = Dataset.from_pandas(df_test1).map(tok_func, batched=True)
bs = 1
epochs = 2
lr = 4.15e-6

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=3)

trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz)#, compute_metrics=compute_metrics
trainer.train();
#pipe = TextClassificationPipeline(model=model, tokenizer=tokz)
#prediction = pipe("The text to predict", return_all_scores=True)
#print(prediction)
preds = trainer.predict(eval_ds).predictions.astype(float)
print(preds)
preds = np.clip(preds, 0, 1)

# Make predictions
#predictions = classifier.predict(X_test)

# Evaluate the model (optional)
#classifier.evaluate(X_test)
submission = df_test.id.copy().to_frame()
submission["prediction"] = np.argmax(preds, axis=1)#classifier.predict(X_test)
#submission = df_test.id.copy().to_frame()
#submission["prediction"] = np.argmax(predictions, axis=1)

#submission["generated"] = submission["generated"].round(1)
submission.to_csv("/kaggle/working/submission.csv", index=False)

TensorFlow version: 2.16.1
KerasNLP version: 0.14.4
/kaggle/input/debertav3base/rust_model.ot
/kaggle/input/debertav3base/spm.model
/kaggle/input/debertav3base/config.json
/kaggle/input/debertav3base/tf_model.h5
/kaggle/input/debertav3base/tokenizer_config.json
/kaggle/input/debertav3base/pytorch_model.bin
/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv
/kaggle/input/debertav3small/spm.model
/kaggle/input/debertav3small/config.json
/kaggle/input/debertav3small/README.md
/kaggle/input/debertav3small/tf_model.h5
/kaggle/input/debertav3small/tokenizer_config.json
/kaggle/input/debertav3small/pytorch_model.bin
/kaggle/input/pii-deberta-models/piirate-training-hall.png
/kaggle/input/pii-deberta-models/piiratefisk.png
/kaggle/input/pii-deberta-models/piirates-on-boat.png
/kaggle/input/pii-deberta-models/hacker-piirates.png
/kaggle/input/pii-deberta-models/piinguinos.png
/

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3030 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'language', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2575
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'language', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 455
    })
})


Map:   0%|          | 0/5195 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of the model checkpoint at /kaggle/input/huggingface-deberta-variants/deberta-base-mnli/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureW

Epoch,Training Loss,Validation Loss
1,1.042000,1.313533
2,0.903500,1.399237


[[-0.64208984  0.06585693  0.12219238]
 [-0.82958984 -0.14208984  0.55029297]
 [ 6.14453125 -2.8359375  -3.31054688]
 ...
 [-1.38378906  2.61523438 -1.65722656]
 [ 6.375      -2.7265625  -3.66796875]
 [-2.54296875 -2.52539062  6.2109375 ]]


In [13]:
sub = pd.read_csv('/kaggle/working/submission.csv')
sub

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,2
2,e98005252c,0
3,58518c10ba,0
4,c32b0d16df,2
...,...,...
5190,5f90dd59b0,2
5191,f357a04e86,1
5192,1f0ea92118,1
5193,0407b48afb,0


In [14]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
#os.rmdir(folder_path)